In [5]:
import numpy as np
from matplotlib import pyplot as plt
import ctypes
from ctypes import POINTER, c_int, c_double, c_void_p, c_char_p
import sys

%matplotlib inline

In [6]:
lib = ctypes.CDLL("./main.so")
lib.parse.restype = c_void_p

In [7]:
def call_func(func, fs, n, h, x0, y0):
    out = np.zeros(n, dtype=np.float64)
    ptr = out.__array_interface__['data'][0]
    f = lib.parse(fs)
    func.argtypes = [c_void_p, c_int, c_double, c_double, c_double, c_void_p]
    func(f, n, h, x0, y0, ptr)
    return out

In [8]:
call_func(lib.euler, 'sin(x)', 10, 0.2, 0.0, 0.0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])